In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
import time 
from pandas import Series
%matplotlib inline

# 定义文件全局变量
path = "/Users/skyamz/Desktop/JDD_Loan_Forecasting_Qualification/initdata/"
generate_path = "/Users/skyamz/Desktop/JDD_Loan_Forecasting_Qualification/generate_data/"


# 加载文件
file_order = pd.read_csv(path + "t_order.csv")
file_click = pd.read_csv(path + "t_click.csv")
file_user = pd.read_csv(path + "t_user.csv")
file_loan = pd.read_csv(path + "t_loan.csv")
file_loan_sum = pd.read_csv(path + "t_loan_sum.csv")

# 格式化时间
file_order['buy_time_format'] = pd.to_datetime(file_order['buy_time'])
file_loan['loan_time_format'] = pd.to_datetime(file_loan['loan_time'])

# 创建索引
file_order = file_order.set_index('buy_time_format')
file_loan = file_loan.set_index('loan_time_format')

#聚合用户订单月份数据
def user_order_month_aggregate(month):
    # 取月份数据
    file_order_month = file_order[month]
    # 重置索引
    file_order_month = file_order_month.reset_index()
    # 统计用户订单某个类目的价格、数量、折扣
    file_order_month_sum = file_order_month.groupby(['uid','cate_id'])['price','qty','discount'].sum()
    # 重置索引
    file_order_month_sum = file_order_month_sum.reset_index()
    # 合并2列
    file_order_month_sum['cate_id_and_count'] = file_order_month_sum['cate_id'].astype('str') + ':' +  file_order_month_sum['qty'].astype('str')
    # 统计用户下单类目
    file_order_month_sum_unique_cate_list = file_order_month_sum.groupby('uid')['cate_id_and_count'].unique()
    # 重置索引
    file_order_month_sum_unique_cate_list = file_order_month_sum_unique_cate_list.reset_index()
    # 统计用户订单全部价格、数量、折扣
    file_order_month_total = file_order_month.groupby(['uid'])['price','qty','discount'].sum()
    # 重置索引
    file_order_month_total = file_order_month_total.reset_index()
    # 将数据进行聚合操作
    user_order_month_aggregate = pd.merge(file_order_month_sum_unique_cate_list,file_order_month_total,how='outer',on='uid')
    return user_order_month_aggregate

# 索引各个月份数据
file_order_month_aggregate_11 = user_order_month_aggregate('2016-11')
file_order_month_aggregate_10 = user_order_month_aggregate('2016-10')
file_order_month_aggregate_09 = user_order_month_aggregate('2016-09')
file_order_month_aggregate_08 = user_order_month_aggregate('2016-08')

# 合并
file_click['pid_param'] = file_click['pid'].astype('str') +'_' + file_click['param'].astype('str')

# 用户点击统计
user_click_count = pd.DataFrame(file_click.groupby(['uid','pid_param'])['uid'].count())

# 重命名统计列
user_click_count.columns = ['click_count']

# 查询某个用户的点击统计
def user_click_count_serch(uid):
   user_click_count_serch = user_click_count.xs(uid, level='uid')
   return user_click_count_serch

# 查询某个参数的用户点击
def pid_and_param_click_count_serch(pid_param):
   pid_and_param_click_count_serch = user_click_count.xs(pid_param, level='pid_param')
   return pid_and_param_click_count_serch

# 查询用户的点击统计
# user_click_count_serch(1)

# 查询某个参数的用户点击
# pid_and_param_click_count_serch('8_1')

# 重置索引,并设置索引为uid
user_click_count_detail = user_click_count.reset_index().set_index('uid')

# 统计用户的总点击次数
user_click_count_total = pd.DataFrame(user_click_count.groupby('uid')['click_count'].sum())

# 重命名列名称
user_click_count_total.columns = ['click_count_total']

# 重置索引
user_click_count_total = user_click_count_total.reset_index()

# 合并2列为字典
user_click_count_detail['param_and_click'] = user_click_count_detail['pid_param'].astype('str') + ':' + user_click_count_detail['click_count'].astype('str')

# 合并分组
user_click_count_detail_aggregate = pd.DataFrame(user_click_count_detail.groupby('uid')['param_and_click'].unique())

# 重置索引
user_click_count_detail_aggregate = user_click_count_detail_aggregate.reset_index()

# 用户点击聚合
user_click_count_aggregate = pd.merge(user_click_count_detail_aggregate,user_click_count_total,how='outer',on='uid')

# 各月用户点击和订单数据汇总
user_click_count_and_order_month_aggregate_08 = pd.merge(user_click_count_detail_aggregate,file_order_month_aggregate_08,how='outer',on='uid')
user_click_count_and_order_month_aggregate_09 = pd.merge(user_click_count_detail_aggregate,file_order_month_aggregate_09,how='outer',on='uid')
user_click_count_and_order_month_aggregate_10 = pd.merge(user_click_count_detail_aggregate,file_order_month_aggregate_10,how='outer',on='uid')
user_click_count_and_order_month_aggregate_11 = pd.merge(user_click_count_detail_aggregate,file_order_month_aggregate_11,how='outer',on='uid')

# 重置索引
user_click_count_detail = user_click_count_detail.reset_index()

# 统计用户最大点击
user_click_count_detail_max = user_click_count_detail.loc[user_click_count_detail.groupby(['uid'])['click_count'].idxmax()].sort_values(by='uid')

# 查询每月订单最大数量
def file_order_month_max(month):
    file_order_month_sum = file_order[month].reset_index().groupby(['uid','cate_id'])['price','qty','discount'].sum().reset_index()
    file_order_month_sum.loc[file_order_month_sum.groupby(['uid'])['qty'].idxmax()].sort_values(by='uid')
    file_order_month_sum_max = file_order_month_sum.loc[file_order_month_sum.groupby(['uid'])['qty'].idxmax()].sort_values(by='uid')
    file_order_month_sum_max.columns = ['uid','cate_id_by_max_qty','price_by_max_qty','qty_max','discount_by_max_qty']
    return file_order_month_sum_max

# 查询点击最多次数和购买数量最大
def click_count_max_and_order_qty_max(month):
    click_count_max_and_order_qty_max = pd.merge(user_click_count_detail_max,file_order_month_max(month),how='outer',on='uid')
    click_count_max_and_order_qty_max = pd.merge(click_count_max_and_order_qty_max,file_user,how='outer',on='uid')
    return click_count_max_and_order_qty_max

# 统计贷款笔数
def loan_count(month):
    file_loan_count = pd.DataFrame(file_loan[month].groupby('uid')['uid'].count())
    file_loan_count.columns = ['loan_count']
    file_loan_count = file_loan_count.reset_index()
    file_loan_sum = pd.DataFrame(file_loan[month].groupby(['uid'])['loan_amount'].sum())
    file_loan_sum.columns = ['loan_sum']
    file_loan_sum = file_loan_sum.reset_index()
    file_loan_count_and_sum = pd.merge(file_loan_count,file_loan_sum,how='outer',on='uid')
    file_loan_count_and_sum['loan_sum_mean'] = file_loan_count_and_sum['loan_sum']/file_loan_count_and_sum['loan_count']
    return file_loan_count_and_sum

#打标用户当月是否借贷
def is_month_loan_user(month):
    loan_month = file_loan[month]
    is_loan_user = loan_month['uid'].unique()
    is_loan_user = pd.DataFrame(Series(is_loan_user))
    is_loan_user.columns = ['uid']
    is_loan_user['is_loan'] = 1
    return is_loan_user

#构建每个月的用户特征
def feature_month_uid(month,prefix_name):
    click_count_max_and_order_qty_max_tmp = click_count_max_and_order_qty_max(month)
    loan_count_tmp = loan_count(month)
    feature_month_uid = pd.merge(click_count_max_and_order_qty_max_tmp,loan_count_tmp,how='outer',on='uid')
    is_month_loan_user_tmp = is_month_loan_user(month)
    feature_month_uid = pd.merge(feature_month_uid,is_month_loan_user_tmp,how='outer',on='uid')
    feature_month_uid['diff'] = feature_month_uid['limit'] - feature_month_uid['loan_sum_mean']
    #one_hot_pid_param = pd.get_dummies(feature_month_uid['pid_param'])
    #one_hot_cate_id_by_max_qty = pd.get_dummies(feature_month_uid['cate_id_by_max_qty'])
    #one_hot_pid_param['uid'] = feature_month_uid['uid']
    #one_hot_cate_id_by_max_qty['uid'] = feature_month_uid['uid']
    #one_hot_all = pd.merge(one_hot_pid_param,one_hot_cate_id_by_max_qty,how = 'outer',on='uid')
    #feature_month_uid = pd.merge(feature_month_uid,one_hot_all,how='outer',on='uid')
    del feature_month_uid['active_date']
    del feature_month_uid['param_and_click']
    del feature_month_uid['pid_param']
    del feature_month_uid['cate_id_by_max_qty']
    feature_month_uid.columns = [col+prefix_name for col in feature_month_uid.columns.values.astype('str')]
    feature_month_uid = feature_month_uid.rename(columns={"uid"+prefix_name:"uid"})
    feature_month_uid = feature_month_uid.where(feature_month_uid.notnull(), 0)
    return feature_month_uid

#只查询贷款贷款
def feature_month_uid_is_loan(month,prefix_name):
    feature_month_uid_is_loan = feature_month_uid(month,prefix_name)
    feature_month_uid_is_loan = feature_month_uid_is_loan[feature_month_uid_is_loan['is_loan'+prefix_name] == 1.0]
    feature_month_uid_is_loan['loan_sum'+prefix_name] = feature_month_uid_is_loan['loan_sum'+prefix_name]
    feature_month_uid_is_loan['limit'+prefix_name] = feature_month_uid_is_loan['limit'+prefix_name]
    feature_month_uid_is_loan['loan_sum_mean'+prefix_name] = feature_month_uid_is_loan['loan_sum_mean'+prefix_name]
    return feature_month_uid_is_loan

# 取各月数据
feature_month_uid_8 = feature_month_uid('2016-08','_8')
feature_month_uid_9 = feature_month_uid('2016-09','_9')
feature_month_uid_10 = feature_month_uid('2016-10','_10')
feature_month_uid_11 = feature_month_uid('2016-11','_11')

feature_month_uid_is_loan_8 = feature_month_uid_is_loan('2016-08','_8')
feature_month_uid_is_loan_9 = feature_month_uid_is_loan('2016-09','_9')
feature_month_uid_is_loan_10 = feature_month_uid_is_loan('2016-10','_10')
feature_month_uid_is_loan_11 = feature_month_uid_is_loan('2016-11','_11')

# 合并每个月数据
feature_month_uid_8_9 = pd.merge(feature_month_uid_8,feature_month_uid_9,how='outer',on='uid')
feature_month_uid_10_11 = pd.merge(feature_month_uid_10,feature_month_uid_11,how='outer',on='uid')
feature_month_uid_all = pd.merge(feature_month_uid_8_9,feature_month_uid_10_11,how='outer',on='uid')

feature_month_uid_is_loan_8_9 = pd.merge(feature_month_uid_is_loan_8,feature_month_uid_is_loan_9,how='outer',on='uid')
feature_month_uid_is_loan_10_11 = pd.merge(feature_month_uid_is_loan_10,feature_month_uid_is_loan_11,how='outer',on='uid')
feature_month_uid_is_loan_all = pd.merge(feature_month_uid_is_loan_8_9,feature_month_uid_is_loan_10_11,how='outer',on='uid')


# 替换所有nan为0
feature_month_uid_all_replace_none = feature_month_uid_all.where(feature_month_uid_all.notnull(), 0)
feature_month_uid_is_loan_all_replace_none = feature_month_uid_is_loan_all.where(feature_month_uid_is_loan_all.notnull(), 0)

# 按uid从小到大排序
feature_month_uid_all_replace_none = feature_month_uid_all_replace_none.sort_values(by='uid')
feature_month_uid_is_loan_all_replace_none = feature_month_uid_is_loan_all_replace_none.sort_values(by='uid')

def build_train_set():
    train_feature_month_uid_all_replace_none = pd.merge(feature_month_uid_is_loan_all_replace_none , file_loan_sum,how='outer',on='uid')
    train_feature_month_uid_all_replace_none = train_feature_month_uid_all_replace_none.where(train_feature_month_uid_all_replace_none.notnull(), 0)
    train_feature_month_uid_all_replace_none['loan_sum'] = train_feature_month_uid_all_replace_none['loan_sum']/10
    train_feature_month_uid_all_replace_none = train_feature_month_uid_all_replace_none[:int(len(train_feature_month_uid_all_replace_none['uid'])*0.8)]
    
    cols = list(train_feature_month_uid_all_replace_none)
    cols.insert(0, cols.pop(cols.index('loan_sum')))
    train_feature_month_uid_all_replace_none = train_feature_month_uid_all_replace_none.loc[train_feature_month_uid_all_replace_none.index[:], cols]
    return train_feature_month_uid_all_replace_none
    
    
def build_test_set():
    test_feature_month_uid_all_replace_none = pd.merge(feature_month_uid_is_loan_all_replace_none , file_loan_sum,how='outer',on='uid')
    test_feature_month_uid_all_replace_none = test_feature_month_uid_all_replace_none.where(test_feature_month_uid_all_replace_none.notnull(), 0)
    test_feature_month_uid_all_replace_none['loan_sum'] = test_feature_month_uid_all_replace_none['loan_sum']/10
    test_feature_month_uid_all_replace_none = test_feature_month_uid_all_replace_none[int(len(test_feature_month_uid_all_replace_none['uid'])*0.8)+1:]
    
    cols = list(test_feature_month_uid_all_replace_none)
    cols.insert(0, cols.pop(cols.index('loan_sum')))
    test_feature_month_uid_all_replace_none = test_feature_month_uid_all_replace_none.loc[test_feature_month_uid_all_replace_none.index[:], cols]
    return test_feature_month_uid_all_replace_none
    
build_train_set = build_train_set()

build_test_set = build_test_set()

# build_train_set.to_csv(generate_path + 'build_train_set.csv')

# build_test_set.to_csv(generate_path + 'build_test_set.csv')

# dtrain = xgb.DMatrix(generate_path + 'build_train_set.libsvm')

# dtest = xgb.DMatrix(generate_path + 'build_test_set.libsvm')

In [7]:
fin_set = pd.merge(feature_month_uid_is_loan_all_replace_none , file_loan_sum,how='outer',on='uid')

In [8]:
fin_set = fin_set.where(fin_set.notnull(), 0)

In [9]:
fin_set

,uid,click_count_8,price_by_max_qty_8,qty_max_8,discount_by_max_qty_8,age_8,sex_8,limit_8,loan_count_8,loan_sum_8,...,discount_by_max_qty_11,age_11,sex_11,limit_11,loan_count_11,loan_sum_11,loan_sum_mean_11,is_loan_11,diff_11,loan_sum
0,4,50.0,14.665733,4.0,11.159862,30.0,1.0,5.292154,2.0,10.445692,...,0.000000,30.0,1.0,5.292154,1.0,4.723017,4.723017,1.0,0.569137,4.723017
1,5,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,21.030112,30.0,2.0,6.292055,1.0,6.584075,6.584075,1.0,-0.292020,6.584075
2,6,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,13.759507,25.0,1.0,6.292055,1.0,6.266691,6.266691,1.0,0.025363,6.266691
3,17,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,30.0,2.0,5.292154,1.0,5.657311,5.657311,1.0,-0.365157,5.657311
4,19,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,50.0,1.0,5.974677,1.0,5.885770,5.885770,1.0,0.088907,5.885770
5,20,29.0,18.710273,8.0,16.992488,35.0,1.0,6.292055,6.0,30.573341,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
6,23,79.0,25.752178,9.0,15.675134,35.0,1.0,5.722768,5.0,19.312973,...,0.000000,35.0,1.0,5.722768,1.0,3.862595,3.862595,1.0,1.860174,3.862595
7,26,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
8,27,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
9,28,33.0,9.793121,3.0,6.589657,25.0,1.0,6.153414,1.0,4.292651,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000


In [27]:
build_train_set.to_csv(generate_path + 'build_train_set.csv')

build_test_set.to_csv(generate_path + 'build_test_set.csv')

In [10]:
fin_set.to_csv(generate_path + 'fin_set.csv')

In [18]:
label, train = np.split(build_train_set, (1, ), axis=1)
dtrain = xgb.DMatrix(train, label=label)

cv, test = np.split(build_test_set, (1, ), axis=1)
dtest = xgb.DMatrix(test)

In [19]:
# 设置参数
param = {'max_depth': 4,
         'eta': 0.3,
         'silent': 0,
         'objective': 'reg:logistic'}

num_round = 5

# 训练模型
bst = xgb.train(param, dtrain, num_round)

In [20]:
# 预测模型
preds = bst.predict(dtest)

In [103]:
len(preds)

7339

In [104]:
len(cv)

7339

In [21]:
a = cv['loan_sum'].tolist()

In [22]:
b = np.array(cv['loan_sum'])

In [23]:
np.array(cv['loan_sum'])

array([ 0.48616016,  0.62126306,  0.56909013, ...,  0.62920545,
        0.70596785,  0.        ])

In [116]:
preds

array([ 0.47956288,  0.58124959,  0.5466401 , ...,  0.58733928,
        0.61771166,  0.10117526], dtype=float32)

In [25]:
for i in b-preds:
    print (i)

0.0065972813913
0.0400134630116
0.0224500336051
-0.101175256073
0.0157190535789
-0.0200817939678
-0.00153925153591
-0.101175256073
-0.0236296704141
-0.101175256073
-0.101175256073
-0.0183907034885
0.0400134630116
0.0256367382051
-0.101175256073
0.0177377846541
0.0256367382051
-0.00435537249396
-0.101175256073
0.0486942318409
-0.101175256073
0.0256367382051
-0.101175256073
0.0268513534789
0.0445130418324
-0.101175256073
0.0157190535789
-0.101175256073
-0.101175256073
0.0501530294307
-0.101175256073
0.0280271853484
-0.00551376937695
-0.101175256073
0.0253356659483
-0.101175256073
0.00302236074015
0.00226450780421
-0.101175256073
-0.101175256073
0.0114947673787
0.0646968413819
-0.101175256073
-0.101175256073
-0.101175256073
-0.00688326207658
-0.101175256073
0.00302236074015
-0.101175256073
-0.101175256073
-0.101175256073
-0.101175256073
0.0340917909116
-0.101175256073
0.0258726644942
-0.101175256073
-0.101175256073
-0.101175256073
-0.101175256073
0.0309974259007
0.0593510654896
0.01867905

0.0162849628896
-0.101175256073
-0.101175256073
0.0200169979784
-0.0342235410885
0.00800056079417
0.0366201784911
-0.101175256073
0.0129897710789
0.0452086671784
-0.101175256073
-0.101175256073
-0.00925541642101
0.0175012168396
0.0190207007983
-0.101175256073
-0.546640098095
0.0179214159913
-0.101175256073
-0.101175256073
-0.101175256073
-0.101175256073
-0.0882596790678
0.0264808457199
0.0117554354245
-0.101175256073
0.075150864606
-0.101175256073
-0.101175256073
-0.101175256073
0.0534532454937
0.022102781681
0.0563070383612
0.00302236074015
-0.0568782712693
0.0261447326405
-0.0178010261397
0.0454195484116
-0.101175256073
-0.000337535476581
-0.101175256073
0.0341338224324
0.0340917909116
-0.053339572721
-0.101175256073
-0.101175256073
0.020016906095
-0.391773223877
-0.101175256073
0.00693075891928
-0.00789543826931
-0.101175256073
-0.101175256073
-0.101175256073
-0.101175256073
-0.101175256073
0.0276127203896
-0.101175256073
0.0580095162819
-0.101175256073
0.0282818121199
-0.1011752560

-0.101175256073
-0.101175256073
0.0179214159913
-0.101175256073
-0.101175256073
-0.101175256073
0.0157190535789
0.0157190535789
0.00693075891928
-0.101175256073
-0.101175256073
0.0481243437397
-0.101175256073
-0.101175256073
0.0608695728483
0.00693075891928
0.0355790710937
0.0256367382051
0.0157190535789
0.0243494384483
-0.101175256073
0.0072908119405
0.0136270503468
0.0328522394007
-0.101175256073
-0.101175256073
0.00693075891928
-0.101175256073
-0.101175256073
0.0196451004983
-0.101175256073
-0.101175256073
-0.101175256073
0.0280271853484
0.0576055967389
-0.101175256073
-0.00240079961029
-0.101175256073
-0.101175256073
-0.101175256073
-0.101175256073
-0.101175256073
-0.479562878609
-0.101175256073
-0.101175256073
0.0355790710937
-0.101175256073
0.0301490299896
-0.101175256073
-0.101175256073
0.0356772995483
-0.101175256073
0.0355790710937
0.0355790710937
-0.101175256073
-0.101175256073
-0.000966081328546
-0.00551376937695
-0.101175256073
-0.101175256073
-0.101175256073
-0.00452912108

In [124]:
np.sqrt((((b-preds)**2).sum())/7339)

0.088182131968947586

In [119]:
b

array([ 0.48616016,  0.62126306,  0.56909013, ...,  0.62920545,
        0.70596785,  0.        ])

In [15]:
b-preds

NameError: name 'b' is not defined

In [2]:
label, train = np.split(build_train_set, (1, ), axis=1)
dtrain = xgb.DMatrix(train, label=label)

#cv, test = np.split(build_test_set, (1, ), axis=1)
dtest = xgb.DMatrix(feature_month_uid_is_loan_all_replace_none)

# 设置参数
param = {'max_depth': 4,
         'eta': 0.3,
         'silent': 0,
         'objective': 'reg:logistic'}

num_round = 5

# 训练模型
bst = xgb.train(param, dtrain, num_round)

preds = bst.predict(dtest)

In [14]:
preds

array([ 0.46537092,  0.60495424,  0.58712924, ...,  0.58733928,
        0.61771166,  0.10117526], dtype=float32)

In [3]:
result = pd.DataFrame(preds)
result.columns=['loan_sum']
result['uid']=feature_month_uid_is_loan_all_replace_none['uid']
result = result[['uid','loan_sum']]
result['loan_sum'] = result['loan_sum'] *10
fin_result = pd.merge(result,file_user,how='outer',on='uid')

In [8]:
for i in fin_result.index:
    if fin_result['loan_sum'][i]>fin_result['limit'][i]:
        fin_result['loan_sum'][i]=fin_result['limit'][i]

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
for i in fin_result.index:
    if fin_result['loan_sum'][i]!=fin_result['loan_sum'][i]:
        fin_result['loan_sum'][i]=0

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
del fin_result['age']
del fin_result['sex']
del fin_result['active_date']
del fin_result['limit']

In [13]:
fin_result.to_csv('fin_result.csv')